# MPIA Arxiv on Deck 2

Contains the steps to produce the paper extractions.

In [1]:
# Imports
import os
from IPython.display import Markdown, display
from tqdm.notebook import tqdm
import warnings
from PIL import Image 

# requires arxiv_on_deck_2

from arxiv_on_deck_2.arxiv2 import (get_new_papers, 
                                    get_paper_from_identifier,
                                    retrieve_document_source, 
                                    get_markdown_badge)
from arxiv_on_deck_2 import (latex, 
                             mpia,
                             highlight_authors_in_list)

# Sometimes images are really big
Image.MAX_IMAGE_PIXELS = 1000000000 

In [2]:
# Some useful definitions.

class AffiliationWarning(UserWarning):
    pass

class AffiliationError(RuntimeError):
    pass

def validation(source: str):
    """Raises error paper during parsing of source file
    
    Allows checks before parsing TeX code.
    
    Raises AffiliationWarning
    """
    check = mpia.affiliation_verifications(source, verbose=True)
    if check is not True:
        raise AffiliationError("mpia.affiliation_verifications: " + check)

        
warnings.simplefilter('always', AffiliationWarning)

In [3]:
# additional CSS to consider. 
# TODO: Current into each exported file. 
#       This should be set once into the webpages directly.
#       With only the class/id definitions in the .md files.

debug_html = """
<style>
#wrap{ overflow:auto; }
#fig1{ background:yellow; width:100%; float:left; padding:5px;  }
#fig2{ background:red; width:50%; float:left; clear:left; padding:5px;  }
#fig3{ background:green; width:50%; float:left; padding:5px;   }
.macros{ background:yellow; visibility:visible;}
h1 {margin: 0 0 0 0;}
mark {background-color:#fff3b6;}
img {object-fit:contain; max-height:250px; display:inline-block; text-align: center;}
</style>
""" 
html = """
<style>
#wrap{ overflow:auto; }
#fig1{ width:100%; float:left; padding: 5px;  }
#fig2{ width:50%; float:left; clear:left; padding: 5px;  }
#fig3{ width:50%; float:left; padding: 5px;  }
.macros{ visibility:hidden; height:0px; }
h1 {margin: 0em 0 0 0;}
mark {background-color:#fff3b6;}
img {object-fit:contain; max-height:250px; display:inline-block; text-align: center;}
</style>
"""

## get list of arxiv paper candidates

We use the MPIA mitarbeiter list webpage from mpia.de to get author names
We then get all new papers from Arxiv and match authors

In [4]:
# get list from MPIA website
# it automatically filters identified non-scientists :func:`mpia.filter_non_scientists`
mpia_authors = mpia.get_mpia_mitarbeiter_list()
new_papers = get_new_papers()
# add manual references
add_paper_refs = []
new_papers.extend([get_paper_from_identifier(k) for k in add_paper_refs])

# select only papers with matching author names and highlight authors
hl_list = [k[0] for k in mpia_authors]

candidates = []
for paperk in new_papers:
    hl_authors = highlight_authors_in_list(paperk['authors'], hl_list)
    matches = [(hl, orig) for hl, orig in zip(hl_authors, paperk['authors']) if 'mark' in hl]
    paperk['authors'] = hl_authors
    if matches:
        candidates.append(paperk)
print("""Arxiv has {0:,d} new papers today""".format(len(new_papers)))        
print("""          {0:,d} with possible author matches""".format(len(candidates)))

Arxiv has 87 new papers today
          40 with possible author matches


# Parse sources and generate relevant outputs

From the candidates, we do the following steps:
* get their tarball from ArXiv (and extract data)
* find the main .tex file: find one with \documentclass{...} (sometimes it's non trivial)
* Check affiliations with :func:`validation`, which uses :func:`mpia.affiliation_verifications`
* If passing the affiliations: we parse the .tex source
   * inject sub-documents into the main (flatten the main document)
   * parse structure, extract information (title, abstract, authors, figures...)
   * handles `\graphicspath` if provided
* Generate the .md document.

In [5]:
documents = []
failed = []
for paper in tqdm(candidates[:-1]):
    paper_id = paper['identifier'].lower().replace('arxiv:', '')
    
    folder = f'tmp_{paper_id}'

    try:
        if not os.path.isdir(folder):
            folder = retrieve_document_source(f"{paper_id}", f'tmp_{paper_id}')
        
        try:
            doc = latex.LatexDocument(folder, validation=validation)    
        except AffiliationError as affilerror:
            msg = f"ArXiv:{paper_id:s} is not an MPIA paper... " + str(affilerror)
            failed.append((paper, "affiliation error: " + str(affilerror) ))
            continue
        
        # Hack because sometimes author parsing does not work well
        if (len(doc.authors) != len(paper['authors'])):
            doc._authors = paper['authors']
        if (doc.abstract) in (None, ''):
            doc._abstract = paper['abstract']
            
        doc.comment = (get_markdown_badge(paper_id) + 
                       "<mark>Appeared on: " + paper['date'] + "</mark> - " +
                       "_" + paper['comments'] + "_")
        doc.highlight_authors_in_list(hl_list)

        full_md = doc.generate_markdown_text()
        
        documents.append((paper_id, full_md))
    except Exception as e:
        warnings.warn(latex.LatexWarning(f"{paper_id:s} did not run properly\n" +
                                         str(e)
                                        ))
        failed.append((paper, "latex error " + str(e)))

  0%|          | 0/39 [00:00<?, ?it/s]

Retrieving document from  https://arxiv.org/e-print/2301.07113


extracting tarball to tmp_2301.07113...

 done.
Retrieving document from  https://arxiv.org/e-print/2301.07114


extracting tarball to tmp_2301.07114...

...

/opt/hostedtoolcache/Python/3.9.16/x64/lib/python3.9/site-packages/IPython/core/interactiveshell.py:3442: LatexWarning: Multiple tex files.

  exec(code_obj, self.user_global_ns, self.user_ns)
/opt/hostedtoolcache/Python/3.9.16/x64/lib/python3.9/site-packages/IPython/core/interactiveshell.py:3442: LatexWarning: Found documentclass in tmp_2301.07651/main.tex

  exec(code_obj, self.user_global_ns, self.user_ns)
/opt/hostedtoolcache/Python/3.9.16/x64/lib/python3.9/site-packages/arxiv_on_deck_2/latex.py:292: LatexWarning: Latex injecting: 'affiliations.tex' from 'tmp_2301.07651/affiliations.tex'
  warnings.warn(LatexWarning(f"Latex injecting: '{ext}' from '{subsource}'"))
/opt/hostedtoolcache/Python/3.9.16/x64/lib/python3.9/site-packages/arxiv_on_deck_2/latex.py:292: LatexWarning: Latex injecting: '6-appendix.tex' from 'tmp_2301.07651/6-appendix.tex'
  warnings.warn(LatexWarning(f"Latex injecting: '{ext}' from '{subsource}'"))
/opt/hostedtoolcache/Python/3.9.16/x64/lib/python3.9/site-packa

extracting tarball to tmp_2301.07688... done.
Retrieving document from  https://arxiv.org/e-print/2301.07692


/opt/hostedtoolcache/Python/3.9.16/x64/lib/python3.9/site-packages/IPython/core/interactiveshell.py:3442: LatexWarning: Multiple tex files.

  exec(code_obj, self.user_global_ns, self.user_ns)
/opt/hostedtoolcache/Python/3.9.16/x64/lib/python3.9/site-packages/IPython/core/interactiveshell.py:3442: LatexWarning: Found documentclass in tmp_2301.07688/ms.tex

  exec(code_obj, self.user_global_ns, self.user_ns)


extracting tarball to tmp_2301.07692...

 done.


### Export the logs

Throughout, we also keep track of the logs per paper. see `logs-{today date}.md` 

In [6]:
import datetime
today = str(datetime.date.today())
logfile = f"_build/html/logs/log-{today}.md"


with open(logfile, 'w') as logs:
    # Success
    logs.write(f'# Arxiv on Deck 2: Logs - {today}\n\n')
    logs.write("""* Arxiv had {0:,d} new papers\n""".format(len(new_papers)))
    logs.write("""    * {0:,d} with possible author matches\n\n""".format(len(candidates)))
    logs.write("## Sucessful papers\n\n")
    display(Markdown("## Successful papers"))
    success = [k[0] for k in documents]
    for candid in candidates:
        if candid['identifier'].split(':')[-1] in success:
            display(candid)
            logs.write(candid.generate_markdown_text() + '\n\n')

    ## failed
    logs.write("## Failed papers\n\n")
    display(Markdown("## Failed papers"))
    failed = sorted(failed, key=lambda x: x[1])
    current_reason = ""
    for paper, reason in failed:
        if 'affiliation' in reason:
            color = 'green'
        else:
            color = 'red'
        data = Markdown(
                paper.generate_markdown_text() + 
                f'\n|<p style="color:{color:s}"> **ERROR** </p>| <p style="color:{color:s}">{reason:s}</p> |'
               )
        if reason != current_reason:
            logs.write(f'### {reason:s} \n\n')
            current_reason = reason
        logs.write(data.data + '\n\n')
        
        # only display here the important errors (all in logs)
        # if color in ('red',):
        display(data)

## Successful papers


|||
|---:|:---|
| [![arXiv](https://img.shields.io/badge/arXiv-arXiv:2301.07116-b31b1b.svg)](https://arxiv.org/abs/arXiv:2301.07116) | **Arkenstone I: A Novel Method for Robustly Capturing High Specific Energy  Outflows In Cosmological Simulations**  |
|| <mark>Matthew C. Smith</mark>, et al. -- incl., <mark>Lars Hernquist</mark>, <mark>Yuan Li</mark> |
|*Appeared on*| *2023-01-19*|
|*Comments*| *Submitted to MNRAS, 26 pages, 15 figures*|
|**Abstract**| Arkenstone is a new model for multiphase, stellar feedback driven galactic winds designed for inclusion in coarse resolution cosmological simulations. In this first paper of a series, we describe the features that allow Arkenstone to properly treat high specific energy wind components and demonstrate them using idealised non-cosmological simulations of a galaxy with a realistic CGM, using the Arepo code. Hot, fast gas phases with low mass loadings are predicted to dominate the energy content of multiphase outflows. In order to treat the huge dynamic range of spatial scales involved in cosmological galaxy formation at feasible computational expense, cosmological volume simulations typically employ a Lagrangian code or else use adaptive mesh refinement with a quasi-Lagrangian refinement strategy. However, it is difficult to inject a high specific energy wind in a Lagrangian scheme without incurring artificial burstiness. Additionally, the low densities inherent to this type of flow result in poor spatial resolution. Arkenstone addresses these issues with a novel scheme for coupling energy into the ISM/CGM transition region which also provides the necessary level of refinement at the base of the wind. In the absence of our improvements, we show that poor spatial resolution near the sonic point of a hot, fast outflow leads to an underestimation of gas acceleration as the wind propagates. We explore the different mechanisms by which low and high specific energy winds can regulate the SFR of galaxies. In future work, we will demonstrate other aspects of the Arkenstone model. |


|||
|---:|:---|
| [![arXiv](https://img.shields.io/badge/arXiv-arXiv:2301.07199-b31b1b.svg)](https://arxiv.org/abs/arXiv:2301.07199) | **The JWST Early Release Science Program for Direct Observations of  Exoplanetary Systems: Best Practices for Data Collection in Cycle 2 and  Beyond**  |
|| Sasha Hinkley, et al. -- incl., <mark>Michael Meyer</mark> |
|*Appeared on*| *2023-01-19*|
|*Comments*| *Not yet submitted for publication. Intended only to be a community resource for JWST Cycle 2 proposals*|
|**Abstract**| We present a set of recommended best practices for JWST data collection for members of the community focussed on the direct imaging and spectroscopy of exoplanetary systems. These findings and recommendations are based on the early analysis of the JWST Early Release Science Program 1386, "High-Contrast Imaging of Exoplanets and Exoplanetary Systems with JWST." Our goal is for this information to be useful for observers in preparation of JWST proposals for Cycle 2 and beyond. In addition to compiling a set of best practices from our ERS program, in a few cases we also draw on the expertise gained within the instrument commissioning programs, as well as include a handful of data processing best practices. We anticipate that this document will be regularly updated and resubmitted to arXiv.org to ensure that we have distributed our knowledge of best-practices for data collection as widely and efficiently as possible. |


|||
|---:|:---|
| [![arXiv](https://img.shields.io/badge/arXiv-arXiv:2301.07366-b31b1b.svg)](https://arxiv.org/abs/arXiv:2301.07366) | **Revisiting HESS J1809$-$193 -- a very-high-energy gamma-ray source in a  fascinating environment**  |
|| <mark>Lars Mohrmann</mark>, Vikas Joshi, Jim Hinton, Stefan Funk |
|*Appeared on*| *2023-01-19*|
|*Comments*| *6 pages, 5 figures. Proceedings of the 7th Heidelberg International Symposium on High-Energy Gamma-Ray Astronomy (Gamma2022)*|
|**Abstract**| HESS J1809$-$193 is one of the unidentified very-high-energy gamma-ray sources in the H.E.S.S. Galactic Plane Survey (HGPS). It is located in a rich environment, with an energetic pulsar and associated X-ray pulsar wind nebula, several supernova remnants, and molecular clouds in the vicinity. Furthermore, HESS J1809$-$193 was recently detected at energies above 56 TeV with HAWC, which makes it a PeVatron candidate, that is, a source capable of accelerating cosmic rays up to PeV energies. We present a new analysis of the TeV gamma-ray emission of HESS J1809$-$193 with H.E.S.S., based on improved analysis techniques. We find that the emission is best described by two components with distinct morphologies and energy spectra. We complement this study with an analysis of Fermi-LAT data in the same region. Finally, taking into account further multi-wavelength data, we interpret our results both in a hadronic and leptonic framework. |

## Failed papers

...


|||
|---:|:---|
| [![arXiv](https://img.shields.io/badge/arXiv-arXiv:2301.07552-b31b1b.svg)](https://arxiv.org/abs/arXiv:2301.07552) | **Correcting Stellar Flare Frequency Distributions Detected by TESS and  Kepler**  |
|| Dong-Yang Gao, <mark>Hui-Gen Liu</mark>, Ming Yang, <mark>Ji-Lin Zhou</mark> |
|*Appeared on*| *2023-01-19*|
|*Comments*| *22 pages, 12 figures, 4 tables. Published in AJ*|
|**Abstract**| The habitability of planets is closely connected with the stellar activity, mainly the frequency of flares and the distribution of flare energy. Kepler and TESS find many flaring stars are detected via precise time-domain photometric data, and the frequency and energy distribution of stellar flares on different types of stars are studied statistically. However, the completeness and observational bias of detected flare events from different missions (e.g. Kepler and TESS) vary a lot. We use a unified data processing and detection method for flares events based on the light curve from Kepler and TESS. Then we perform injection and recovery tests in the original light curve of each star for each flare event to correct the completeness and energy of flares. Three samples of flaring stars are selected from Kepler and TESS, with rotating periods from 1 to $\sim$ 5 days. Adopting a hot-blackbody assumption, our results show that the cumulative flare frequency distributions (FFDs) of the same stars in Kepler and TESS bands tend to be consistent after correction, revealing a more natural flaring frequency and energy distribution. Our results also extend the low-energy limit in cumulative FFD fitting to $10^{31.5-33}$ erg on different types of stars. For solar-type stars, the average power-law index of cumulative FFD ($\alpha_{\rm cum}$) is $-0.84$, which indicates that low-energy flares contribute less to the total flare energy. With a piecewise correlation between $\alpha_{\rm cum}$ and $T_{\rm eff}$, $\alpha_{\rm cum}$ first rises with $T_{\rm eff}$ from M2 to K1 stars, then slightly decreases for stars hotter than K1. |
|<p style="color:green"> **ERROR** </p>| <p style="color:green">affiliation error: mpia.affiliation_verifications: 'Heidelberg' keyword not found.</p> |


|||
|---:|:---|
| [![arXiv](https://img.shields.io/badge/arXiv-arXiv:2301.07692-b31b1b.svg)](https://arxiv.org/abs/arXiv:2301.07692) | **The imprint of bursty star formation on alpha-element abundance patterns  in Milky Way-like galaxies**  |
|| Hanna Parul, et al. -- incl., <mark>Andrew Wetzel</mark> |
|*Appeared on*| *2023-01-19*|
|*Comments*| *15 pages; Accepted 17 January 2023 in MNRAS*|
|**Abstract**| Milky Way-mass galaxies in the FIRE-2 simulations demonstrate two main modes of star formation. At high redshifts star formation occurs in a series of short and intense bursts, while at low redshifts star formation proceeds at a steady rate with a transition from one mode to another at times ranging from 3 to 7 Gyr ago for different galaxies. We analyse how the mode of star formation affects iron and alpha-element abundance. We find that the early bursty regime imprints a measurable pattern in stellar elemental abundances in the form of a "sideways chevron" shape on the [Fe/H] - [O/Fe] plane and the scatter in [O/Fe] at a given stellar age is higher than when a galaxy is in the steady regime. That suggests that the evolution of [O/Fe] scatter with age provides an estimate of the end of the bursty phase. We investigate the feasibility of observing of this effect by adding mock observational errors to a simulated stellar survey and find that the transition between the bursty and steady phase should be detectable in the Milky Way, although larger observational uncertainties make the transition shallower. We apply our method to observations of the Milky Way from the Second APOKASC Catalog and estimate that the transition to steady star formation in the Milky Way happened 7-8 Gyrs ago, earlier than transition times measured in the simulations. |
|<p style="color:green"> **ERROR** </p>| <p style="color:green">affiliation error: mpia.affiliation_verifications: 'Heidelberg' keyword not found.</p> |


|||
|---:|:---|
| [![arXiv](https://img.shields.io/badge/arXiv-arXiv:2301.07186-b31b1b.svg)](https://arxiv.org/abs/arXiv:2301.07186) | **ABYSS I: Targeting strategy for APOGEE & BOSS young star survey in  SDSS-V**  |
|| Marina Kounkel, et al. -- incl., <mark>Eleonora Zari</mark>, <mark>Jesús Hernández</mark> |
|*Appeared on*| *2023-01-19*|
|*Comments*| *18 pages, 12 pages. Submitted to ApJS. Part of SDSS DR18*|
|**Abstract**| The fifth iteration of the Sloan Digital Sky Survey (SDSS-V) is set to obtain optical and near-infrared spectra of $\sim$5 million stars of all ages and masses throughout the Milky Way. As a part of these efforts, APOGEE & BOSS Young Star Survey (ABYSS) will observe $\sim10^5$ stars with ages $<$30 Myr that have been selected using a set of homogeneous selection functions that make use of different tracers of youth. The ABYSS targeting strategy we describe in this paper is aimed to provide the largest spectroscopic census of young stars to-date. It consists of 8 different types of selection criteria that take the position on the HR diagram, infrared excess, variability, as well as the position in phase space in consideration. The resulting catalog of $\sim$200,000 sources (of which a half are expected to be observed) provides representative coverage of the young Galaxy, including both nearby diffuse associations as well as more distant massive complexes, reaching towards the inner Galaxy and the Galactic center. |
|<p style="color:red"> **ERROR** </p>| <p style="color:red">latex error Could not find figure h</p> |


|||
|---:|:---|
| [![arXiv](https://img.shields.io/badge/arXiv-arXiv:2301.07331-b31b1b.svg)](https://arxiv.org/abs/arXiv:2301.07331) | **HLC2: a highly efficient cross-matching framework for large astronomical  catalogues on heterogeneous computing environments**  |
|| <mark>Yajie Zhang</mark>, et al. -- incl., <mark>Kun Li</mark> |
|*Appeared on*| *2023-01-19*|
|*Comments*| *Accepted for publication in Monthly Notices of the Royal Astronomical Society*|
|**Abstract**| Cross-matching operation, which is to find corresponding data for the same celestial object or region from multiple catalogues,is indispensable to astronomical data analysis and research. Due to the large amount of astronomical catalogues generated by the ongoing and next-generation large-scale sky surveys, the time complexity of the cross-matching is increasing dramatically. Heterogeneous computing environments provide a theoretical possibility to accelerate the cross-matching, but the performance advantages of heterogeneous computing resources have not been fully utilized. To meet the challenge of cross-matching for substantial increasing amount of astronomical observation data, this paper proposes Heterogeneous-computing-enabled Large Catalogue Cross-matcher (HLC2), a high-performance cross-matching framework based on spherical position deviation on CPU-GPU heterogeneous computing platforms. It supports scalable and flexible cross-matching and can be directly applied to the fusion of large astronomical cataloguesfrom survey missions and astronomical data centres. A performance estimation model is proposed to locate the performance bottlenecks and guide the optimizations. A two-level partitioning strategy is designed to generate an optimized data placement according to the positions of celestial objects to increase throughput. To make HLC2 a more adaptive solution, the architecture-aware task splitting, thread parallelization, and concurrent scheduling strategies are designed and integrated. Moreover, a novel quad-direction strategy is proposed for the boundary problem to effectively balance performance and completeness. We have experimentally evaluated HLC2 using public released catalogue data. Experiments demonstrate that HLC2 scales well on different sizes of catalogues and the cross-matching speed is significantly improved compared to the state-of-the-art cross-matchers. |
|<p style="color:red"> **ERROR** </p>| <p style="color:red">latex error not a gzip file</p> |


|||
|---:|:---|
| [![arXiv](https://img.shields.io/badge/arXiv-arXiv:2301.07647-b31b1b.svg)](https://arxiv.org/abs/arXiv:2301.07647) | **Solaris: A Focused Solar Polar Discovery-class Mission to achieve the  Highest Priority Heliophysics Science Now**  |
|| Donald M. Hassler, et al. -- incl., <mark>Lisa Upton</mark>, <mark>Heather Alison Elliott</mark>, <mark>Jon Linker</mark>, <mark>Junwei Zhao</mark> |
|*Appeared on*| *2023-01-19*|
|*Comments*| *This White Paper was submitted in 2022 to the United States National Academies Solar and Space Physics (Heliophysics) Decadal Survey*|
|**Abstract**| Solaris is a transformative Solar Polar Discovery-class mission concept to address crucial outstanding questions that can only be answered from a polar vantage. Solaris will image the Sun's poles from ~75 degree latitude, providing new insight into the workings of the solar dynamo and the solar cycle, which are at the foundation of our understanding of space weather and space climate. Solaris will also provide enabling observations for improved space weather research, modeling and prediction, revealing a unique, new view of the corona, coronal dynamics and CME eruptions from above. |
|<p style="color:red"> **ERROR** </p>| <p style="color:red">latex error not a gzip file</p> |

## Export documents

We now write the .md files and export relevant images

In [7]:
def export_markdown_summary(md: str, md_fname:str, directory: str):
    """Export MD document and associated relevant images"""
    import os
    import shutil
    import re

    if (os.path.exists(directory) and not os.path.isdir(directory)):
        raise RuntimeError(f"a non-directory file exists with name {directory:s}")

    if (not os.path.exists(directory)):
        print(f"creating directory {directory:s}")
        os.mkdir(directory)

    fig_fnames = (re.compile(r'\[Fig.*\]\((.*)\)').findall(md) + 
                  re.compile(r'\<img src="([^>\s]*)"[^>]*/>').findall(md))
    for fname in fig_fnames:
        if 'http' in fname:
            # No need to copy online figures
            continue
        destdir = os.path.join(directory, os.path.dirname(fname))
        destfname = os.path.join(destdir, os.path.basename(fname))
        try:
            os.makedirs(destdir)
        except FileExistsError:
            pass
        shutil.copy(fname, destfname)
    with open(os.path.join(directory, md_fname), 'w') as fout:
        fout.write(md)
    print("exported in ", os.path.join(directory, md_fname))
    [print("    + " + os.path.join(directory,fk)) for fk in fig_fnames]

In [8]:
for paper_id, md in documents:
    export_markdown_summary(md, f"{paper_id:s}.md", '_build/html/')

exported in  _build/html/2301.07116.md
    + _build/html/tmp_2301.07116/./figs/slice_with_rcell_subsonic_5mdisk_300.png
    + _build/html/tmp_2301.07116/./figs/profiles_subsonic_5mdisk_more_quants.png
    + _build/html/tmp_2301.07116/./figs/mflows_subsonic_5mdisk_with_nowind.png
exported in  _build/html/2301.07199.md
exported in  _build/html/2301.07366.md
    + _build/html/tmp_2301.07366/./img/sign_maps_models_prelim.png
    + _build/html/tmp_2301.07366/./img/flux_map_prelim.png
    + _build/html/tmp_2301.07366/./img/sign_maps_fermi_prelim.png


## Display the papers

Not necessary but allows for a quick check.

In [9]:
[display(Markdown(k[1])) for k in documents];

<div class="macros" style="visibility:hidden;">
$\newcommand{\ensuremath}{}$
$\newcommand{\xspace}{}$
$\newcommand{\object}[1]{\texttt{#1}}$
$\newcommand{\farcs}{{.}''}$
$\newcommand{\farcm}{{.}'}$
$\newcommand{\arcsec}{''}$
$\newcommand{\arcmin}{'}$
$\newcommand{\ion}[2]{#1#2}$
$\newcommand{\textsc}[1]{\textrm{#1}}$
$\newcommand{\hl}[1]{\textrm{#1}}$
$\newcommand{\kms}{\mathrm{km s^{-1}}}$
$\newcommand{\Msun}{\mathrm{M_\odot}}$
$\newcommand{\lowspec}{\textit{LowSpec}}$
$\newcommand{\highspec}{\textit{HighSpec}}$
$\newcommand{\highark}{\textit{HighSpecArk}}$
$\newcommand{\medark}{\textit{MedSpecArk}}$
$\newcommand{\subfive}{fiducial}$
$\newcommand{\thebibliography}{\DeclareRobustCommand{\VAN}[3]{##3}\VANthebibliography}$</div>

<div class="macros" style="visibility:hidden;">
$\newcommand{$\ensuremath$}{}$
$\newcommand{$\xspace$}{}$
$\newcommand{$\object$}[1]{\texttt{#1}}$
$\newcommand{$\farcs$}{{.}''}$
$\newcommand{$\farcm$}{{.}'}$
$\newcommand{$\arcsec$}{''}$
$\newcommand{$\arcmin$}{'}$
$\newcommand{$\ion$}[2]{#1#2}$
$\newcommand{$\textsc$}[1]{\textrm{#1}}$
$\newcommand{$\hl$}[1]{\textrm{#1}}$
$\newcommand{$\kms$}{\mathrm{km s^{-1}}}$
$\newcommand{$\Msun$}{\mathrm{M_\odot}}$
$\newcommand{$\lowspec$}{\textit{LowSpec}}$
$\newcommand{$\highspec$}{\textit{HighSpec}}$
$\newcommand{$\highark$}{\textit{HighSpecArk}}$
$\newcommand{$\medark$}{\textit{MedSpecArk}}$
$\newcommand{$\subfive$}{fiducial}$
$\newcommand{$\thebibliography$}{\DeclareRobustCommand{\VAN}[3]{##3}\VANthebibliography}$</div>



<div id="title">

# Arkenstone I: A Novel Method for Robustly Capturing High Specific Energy Outflows In Cosmological Simulations

</div>
<div id="comments">

[![arXiv](https://img.shields.io/badge/arXiv-2301.07116-b31b1b.svg)](https://arxiv.org/abs/2301.07116)<mark>Appeared on: 2023-01-19</mark> - _Submitted to MNRAS, 26 pages, 15 figures_

</div>
<div id="authors">

<mark><mark>Matthew C. Smith</mark></mark>, et al. -- incl., <mark><mark>Lars Hernquist</mark></mark>, <mark><mark>Yuan Li</mark></mark>

</div>
<div id="abstract">

**Abstract:** $\textsc${Arkenstone}is a new model for multiphase, stellar feedback driven galactic winds designed for inclusion in coarse resolution cosmological simulations. In this first paper of a series, we describe the features that allow$\textsc${Arkenstone}to properly treat high specific energy wind components and demonstrate them using idealised non-cosmological simulations of a galaxy with a realistic CGM, using the$\textsc${Arepo}code. Hot, fast gas phases with low mass loadings are predicted to dominate the energy content of multiphase outflows. In order to treat the huge dynamic range of spatial scales involved in cosmological galaxy formation at feasible computational expense,cosmological volume simulations typically employ a Lagrangian code or else use adaptive mesh refinement with a quasi-Lagrangian refinement strategy.However, it is difficult to inject a high specific energy wind in a Lagrangian scheme without incurring artificial burstiness. Additionally, the low densities inherent to this type of flow result in poor spatial resolution.$\textsc${Arkenstone}addresses these issues with a novel scheme for coupling energy into the ISM/CGM transition region which also provides the necessary level of refinement at the base of the wind. In the absence of our improvements, we show that poor spatial resolution near the sonic point of a hot, fast outflow leads to an underestimation of gas acceleration as the wind propagates. We explore the different mechanisms by which low and high specific energy winds can regulate the SFR of galaxies. In future work, we will demonstrate other aspects of the$\textsc${Arkenstone}model.

</div>

<div id="div_fig1">

<img src="tmp_2301.07116/./figs/slice_with_rcell_subsonic_5mdisk_300.png" alt="Fig8" width="100%"/>

**Figure 8. -** Slices through the domain for the $\subfive$  simulations at 1.5 Gyr.
The slices cut vertically through the galaxy and the wind. From left to right,
we show the simulations with the $\lowspec$ , $\highspec$  and $\highark$  models.
From top to bottom, we show gas density, temperature, radial velocity
and cell radius. The dashed circles indicate 0.1, 0.2, 0.5 and 1 $r_{200}$.
The $\lowspec$  wind produces a narrow column of cold, dense gas which is
already infalling by 1.5 Gyr, while a low altitude fountain is present.
The $\highspec$  and $\highark$  winds are fast, hot and of lower density
than the surrounding CGM. They fill a larger volume than the $\lowspec$ 
wind. The higher spatial resolution provided by the refinement scheme used
in $\highark$  is evident. (*fig:slice_subsonic_5mdisk*)

</div>
<div id="div_fig2">

<img src="tmp_2301.07116/./figs/profiles_subsonic_5mdisk_more_quants.png" alt="Fig11" width="100%"/>

**Figure 11. -** For the fiducial simulations,
profiles of various quantities as a function of galactocentric radius
measured within a bicone of opening angle $10\degree$ orientated along
the rotational axis of the disc/CGM. Different columns show different
times throughout the simulation. In the top row, we plot density.
The horizontal grey dotted line indicates the wind particle recoupling threshold
density. For reference, we indicate the $\rho \propto r^{-2}$ slope of the
asymptotic
\citetalias{Chevalier1985} solution (with arbitrary normalisation).
The second row shows pressure, with an indication of the $P \propto r^{-10/3}$
slope of \citetalias{Chevalier1985}.
The third row shows sound speed (dashed line) and radial
velocity (solid line).
The bottom row shows the cell radius.
In the $\highark$  simulation, the wind rapidly accelerates as it flows outwards,
passing through a sonic point within $\sim 1 \mathrm{kpc}$. The $\highspec$  simulation
has much coarser spatial resolution. The acceleration is much more gradual and the
sonic point does not occur until the galactocentric radius is resolved by several cells,
which occurs much further out.
 (*fig:profiles_subsonic_5mdisk*)

</div>
<div id="div_fig3">

<img src="tmp_2301.07116/./figs/mflows_subsonic_5mdisk_with_nowind.png" alt="Fig9" width="100%"/>

**Figure 9. -** SFR, mass inflow and outflow rates through spheres
at various radii for the $\subfive$  simulations. Rows from top to bottom
show the no wind, $\lowspec$ , $\highspec$  and $\highark$  simulations,
respectively.
Columns from left to right show measurements at 0.1, 0.2, 0.5 and
1 $r_{200}$, respectively. For a given simulation, we repeat the
SFR in each column for reference. The $\lowspec$  simulation
reduces the SFR significantly relative to the no wind case by
ejecting large quantities of the ISM. This material is kept circulating
in a low altitude fountain flow. By contrast, the $\highspec$  and $\highark$ 
simulations are far more efficient at suppressing inflows from the CGM. This
gradually leads to a reduction in SFR as the supply of star forming gas is
used up without replenishment. (*fig:mflows_subsonic_5mdisk*)

</div>

<div class="macros" style="visibility:hidden;">
$\newcommand{\ensuremath}{}$
$\newcommand{\xspace}{}$
$\newcommand{\object}[1]{\texttt{#1}}$
$\newcommand{\farcs}{{.}''}$
$\newcommand{\farcm}{{.}'}$
$\newcommand{\arcsec}{''}$
$\newcommand{\arcmin}{'}$
$\newcommand{\ion}[2]{#1#2}$
$\newcommand{\textsc}[1]{\textrm{#1}}$
$\newcommand{\hl}[1]{\textrm{#1}}$
$\newcommand{\vdag}{(v)^\dagger}$
$\newcommand$
$\newcommand$</div>

<div class="macros" style="visibility:hidden;">
$\newcommand{$\ensuremath$}{}$
$\newcommand{$\xspace$}{}$
$\newcommand{$\object$}[1]{\texttt{#1}}$
$\newcommand{$\farcs$}{{.}''}$
$\newcommand{$\farcm$}{{.}'}$
$\newcommand{$\arcsec$}{''}$
$\newcommand{$\arcmin$}{'}$
$\newcommand{$\ion$}[2]{#1#2}$
$\newcommand{$\textsc$}[1]{\textrm{#1}}$
$\newcommand{$\hl$}[1]{\textrm{#1}}$
$\newcommand{$\vdag$}{(v)^\dagger}$
$\newcommand$
$\newcommand$</div>



<div id="title">

# The JWST Early Release Science Program for Direct Observations of Exoplanetary Systems: Best Practices for Data Collection in Cycle 2 and Beyond

</div>
<div id="comments">

[![arXiv](https://img.shields.io/badge/arXiv-2301.07199-b31b1b.svg)](https://arxiv.org/abs/2301.07199)<mark>Appeared on: 2023-01-19</mark> - _Not yet submitted for publication. Intended only to be a community resource for JWST Cycle 2 proposals_

</div>
<div id="authors">

Sasha Hinkley, et al. -- incl., <mark><mark>Michael Meyer</mark></mark>

</div>
<div id="abstract">

**Abstract:** We present a set of recommended best practices for JWST data collection for members of the community focussed on the direct imaging and spectroscopy of exoplanetary systems. These findings and recommendations are based on the early analysis of the JWST Early Release Science Program 1386, "High-Contrast Imaging of Exoplanets and Exoplanetary Systems with JWST."  Our goal is for this information to be useful for observers in preparation of JWST proposals for Cycle 2 and beyond.  In addition to compiling a set of best practices from our ERS program, in a few cases we also draw on the expertise gained within the instrument commissioning programs, as well as include a handful of data processing best practices. We anticipate that this document will be regularly updated and resubmitted to the\textit{arXiv}to ensure that we have distributed our knowledge of best-practices for data collection as widely and efficiently as possible.

</div>



<div class="macros" style="visibility:hidden;">
$\newcommand{\ensuremath}{}$
$\newcommand{\xspace}{}$
$\newcommand{\object}[1]{\texttt{#1}}$
$\newcommand{\farcs}{{.}''}$
$\newcommand{\farcm}{{.}'}$
$\newcommand{\arcsec}{''}$
$\newcommand{\arcmin}{'}$
$\newcommand{\ion}[2]{#1#2}$
$\newcommand{\textsc}[1]{\textrm{#1}}$
$\newcommand{\hl}[1]{\textrm{#1}}$
$\newcommand{\hess}{H.E.S.S.\xspace}$
$\newcommand{\fermilat}{\emph{Fermi}-LAT\xspace}$
$\newcommand{\gam}{\gamma\xspace}$
$\newcommand{\hessj}{HESS~J1809-193\xspace}$
$\newcommand{\psrj}{PSR~J1809-1917\xspace}$
$\newcommand{\psrjother}{PSR~J1811-1925\xspace}$
$\newcommand{\snrN}{G011.1+00.1\xspace}$
$\newcommand{\snrS}{G011.0-00.0\xspace}$
$\newcommand{\fermijten}{J1810.3-1925e\xspace}$
$\newcommand{\fermijeleven}{J1811.5-1925\xspace}$
$\newcommand{\}{NAT}$
$\newcommand{\@}{citea}$</div>

<div class="macros" style="visibility:hidden;">
$\newcommand{$\ensuremath$}{}$
$\newcommand{$\xspace$}{}$
$\newcommand{$\object$}[1]{\texttt{#1}}$
$\newcommand{$\farcs$}{{.}''}$
$\newcommand{$\farcm$}{{.}'}$
$\newcommand{$\arcsec$}{''}$
$\newcommand{$\arcmin$}{'}$
$\newcommand{$\ion$}[2]{#1#2}$
$\newcommand{$\textsc$}[1]{\textrm{#1}}$
$\newcommand{$\hl$}[1]{\textrm{#1}}$
$\newcommand{$\hess$}{H.E.S.S.$\xspace$}$
$\newcommand{$\fermilat$}{\emph{Fermi}-LAT$\xspace$}$
$\newcommand{$\gam$}{$\gam$ma$\xspace$}$
$\newcommand{$\hess$j}{HESS~J1809-193$\xspace$}$
$\newcommand{$\psrj$}{PSR~J1809-1917$\xspace$}$
$\newcommand{$\psrj$other}{PSR~J1811-1925$\xspace$}$
$\newcommand{$\snrN$}{G011.1+00.1$\xspace$}$
$\newcommand{$\snrS$}{G011.0-00.0$\xspace$}$
$\newcommand{$\fermijten$}{J1810.3-1925e$\xspace$}$
$\newcommand{$\fermijeleven$}{J1811.5-1925$\xspace$}$
$\newcommand{\}{NAT}$
$\newcommand{\@}{citea}$</div>



<div id="title">

#  --- a very-high-energy gamma-ray source in a fascinating environment

</div>
<div id="comments">

[![arXiv](https://img.shields.io/badge/arXiv-2301.07366-b31b1b.svg)](https://arxiv.org/abs/2301.07366)<mark>Appeared on: 2023-01-19</mark> - _6 pages, 5 figures. Proceedings of the 7th Heidelberg International Symposium on High-Energy Gamma-Ray Astronomy (Gamma2022)_

</div>
<div id="authors">

, V. Joshi, , J. Hinton,  and S. Funk,  for the $\hess$ Collaboration

</div>
<div id="abstract">

**Abstract:** $\hess$jis one of the unidentified very-high-energy gamma-ray sources in the$\hess$Galactic Plane Survey (HGPS).  It is located in a rich environment, with an energetic pulsar and associated X-ray pulsar wind nebula, several supernova remnants, and molecular clouds in the vicinity.  Furthermore,$\hess$jwas recently detected at energies above 56 TeV with HAWC, which makes it a PeVatron candidate, that is, a source capable of accelerating cosmic rays up to PeV energies.  We present a new analysis of the TeV gamma-ray emission of$\hess$jwith$\hess$, based on improved analysis techniques.  We find that the emission is best described by two components with distinct morphologies and energy spectra.  We complement this study with an analysis of Fermi-LAT data in the same region.  Finally, taking into account further multi-wavelength data, we interpret our results both in a hadronic and leptonic framework.

</div>

<div id="div_fig1">

<img src="tmp_2301.07366/./img/sign_maps_models_prelim.png" alt="Fig2" width="100%"/>

**Figure 2. -** $\hess$ significance maps for $\hess$j.
    Panel (a) shows the pre-modelling map, whereas panels (b) and (c) show the residual significance map for the 1-component and the 2-component model, respectively.
    White dashed circles denote regions excluded from the analysis.
   (*fig:sign_maps_hess*)

</div>
<div id="div_fig2">

<img src="tmp_2301.07366/./img/flux_map_prelim.png" alt="Fig1" width="100%"/>

**Figure 1. -** 
    Map showing the $\gam$-ray flux above \SI{0.27}{TeV} from $\hess$j.
    (a) full region.
    (b) zoom-in on core region.
    The position of $\psrj$ is marked with a black triangle, cyan circles denote the positions of SNRs.
    The green/purple dot and lines display the position and extent of the two components (A/B) of $\hess$j(cf. also Fig. \ref{fig:sign_maps_hess}).
    The grey dashed line marks the Galactic plane.
   (*fig:flux_maps*)

</div>
<div id="div_fig3">

<img src="tmp_2301.07366/./img/sign_maps_fermi_prelim.png" alt="Fig3" width="100%"/>

**Figure 3. -** 
    Significance maps for the $\fermilat$ analysis.
    (a) Pre-modelling map.
    (b) With $\fermijeleven$ in the model.
    (c) With $\fermijeleven$ and $\fermijten$ in the model.
    The two components of $\hess$j are displayed as well.
    The grey dashed line marks the Galactic plane.
   (*fig:sign_maps_fermi*)

</div>

# Create HTML index

In [10]:
from datetime import datetime, timedelta, timezone
from glob import glob

files = glob('_build/html/*.md')
days = 7
now = datetime.today()
res = []
for fk in files:
    stat_result = os.stat(fk).st_ctime
    modified = datetime.fromtimestamp(stat_result, tz=timezone.utc).replace(tzinfo=None)
    delta = now.today() - modified
    if delta <= timedelta(days=days):
        res.append((delta.seconds, fk))
res = [k[1] for k in reversed(sorted(res, key=lambda x:x[1]))]
npub = len(res)
print(len(res), f" publications in the last {days:d} days.")
[ print('\t', k) for k in res ];

175  publications in the last 7 days.
	 _build/html/2301.07366.md
	 _build/html/2301.07199.md
	 _build/html/2301.07116.md
	 _build/html/2301.06873.md
	 _build/html/2301.06819.md
	 _build/html/2301.06575.md
	 _build/html/2301.05720.md
	 _build/html/2301.05718.md
	 _build/html/2301.05034.md
	 _build/html/2301.04656.md
	 _build/html/2301.04533.md
	 _build/html/2301.04442.md
	 _build/html/2301.03811.md
	 _build/html/2301.03689.md
	 _build/html/2301.03387.md
	 _build/html/2301.03209.md
	 _build/html/2301.02671.md
	 _build/html/2301.02477.md
	 _build/html/2301.02313.md
	 _build/html/2301.02265.md
	 _build/html/2301.01937.md
	 _build/html/2301.01895.md
	 _build/html/2301.01684.md
	 _build/html/2301.01306.md
	 _build/html/2301.00881.md
	 _build/html/2301.00854.md
	 _build/html/2301.00578.md
	 _build/html/2212.12039.md
	 _build/html/2212.11991.md
	 _build/html/2212.11425.md
	 _build/html/2212.11177.md
	 _build/html/2212.10804.md
	 _build/html/2212.10512.md
	 _build/html/2212.09766.md
	 _build/h

In [11]:
import datetime
from glob import glob

def get_last_n_days(lst, days=1):
    """ Get the documents from the last n days """
    sorted_lst = sorted(lst, key=lambda x: x[1], reverse=True)
    for fname, date in sorted_lst:
        if date >= str(datetime.date.today() - datetime.timedelta(days=days)):
            yield fname

def extract_appearance_dates(lst_file):
    dates = []

    def get_date(line):
        return line\
            .split('Appeared on:')[-1]\
            .split('</mark>')[0].strip()

    for fname in lst:
        with open(fname, 'r') as f:
            found_date = False
            for line in f:
                if not found_date:
                    if "Appeared on" in line:
                        found_date = True
                        dates.append((fname, get_date(line)))
                else:
                    break
    return dates

from glob import glob
lst = glob('_build/html/*md')
days = 7
dates = extract_appearance_dates(lst)
res = list(get_last_n_days(dates, days))
npub = len(res)
print(len(res), f" publications in the last {days:d} days.")

12  publications in the last 7 days.


In [12]:
def create_carousel(npub=4):
    """ Generate the HTML code for a carousel with `npub` slides """
    carousel = ["""<section class="carousel" aria-label="Gallery">""",
                """  <ol class="carousel__viewport">""",
    ]
    for k in range(1, npub + 1):
        prev_ = k - 1
        next_ = k + 1
        if prev_ <= 0:
            prev_ = npub
        if next_ > npub:
            next_ = 1
        text  = f"""    <li id="carousel__slide{k}" tabindex="0" class="carousel__slide">\n"""
        text += f"""       <div class="carousel__snapper">\n"""
        text += f"""         <a href="#carousel__slide{prev_}" class="carousel__prev">Go to previous slide</a>\n"""
        text += f"""         <a href="#carousel__slide{next_}" class="carousel__next">Go to next slide</a>\n"""
        text += f"""         <div id="slide{k}_content" class="md_view" >Content {k}</div>\n"""
        text += f"""       </div>\n"""
        text += f"""    </li>"""
        carousel.append(text)

    carousel.extend([
        """  </ol>""",
        """  <aside class="carousel__navigation">""",
        """    <ol class="carousel__navigation-list">"""])

    for k in range(1, npub + 1):
        text  = f"""      <li class="carousel__navigation-item">\n"""
        text += f"""        <a href="#carousel__slide{k}" class="carousel__navigation-button">Go to {k}</a>\n"""
        text += f"""      </li>"""
        carousel.append(text)
    carousel.extend(["""    </ol>""", """  </aside>""", """</section>"""])

    return '\n'.join(carousel)

In [13]:
carousel = create_carousel(npub)
docs = ', '.join(['"{0:s}"'.format(k.split('/')[-1]) for k in res])
slides = ', '.join([f'"slide{k}_content"' for k in range(1, npub + 1)])

script = f"""
const docs = [{docs}]

const slides = [{slides}]
""" + """
async function run() {
    for (let i = 0; i < docs.length; i++) {
        let file = await fetch(docs[i]);
        let text = await file.text()
        document.getElementById(slides[i]).innerHTML =
            marked.parse(text);
    }
    hljs.highlightAll();
}
run()
"""

page = f"""<!doctype html>
<html lang="en">

<head>
  <meta charset="utf-8">
  <meta name="viewport" content="width=device-width, initial-scale=1">
  <!-- Bootstrap CSS -->
  <link href="https://cdn.jsdelivr.net/npm/bootstrap@5.0.2/dist/css/bootstrap.min.css" rel="stylesheet"
   integrity="sha384-EVSTQN3/azprG1Anm3QDgpJLIm9Nao0Yz1ztcQTwFspd3yD65VohhpuuCOmLASjC" crossorigin="anonymous">
  <!-- highlight.js CSS -->
  <link rel="stylesheet" href="https://cdnjs.cloudflare.com/ajax/libs/highlight.js/11.1.0/styles/default.min.css">
  <!-- Mathjax 3 -->
  <script type="text/javascript" id="MathJax-config" src="mathjax_config.js"> </script>
  <script type="text/javascript" id="MathJax-script" async 
    src="https://cdn.jsdelivr.net/npm/mathjax@3/es5/tex-mml-chtml.js">
  </script>
  <link rel="stylesheet" href="index_carousel.css">
  <link rel="icon" type="image/x-icon" href="https://www.mpia.de/assets/touch-icon-32x32-a66937bcebc4e8894ebff1f41a366c7c7220fd97a38869ee0f2db65a9f59b6c1.png">
  <title>MPIA Arxiv on deck!</title>
</head>

<body>
  <div id="header"> <img src="header_banner.png" width="100%"></div>
  <div id="suptitle"> 7-day archives </div>
  <div id="info">
    <img src="https://pngimg.com/uploads/github/github_PNG58.png" height=30rem></img>
    <a href=https://github.com/mpi-astronomy/arxiv_display style="color:black;">github/mpi-astronomy/arxiv_display</a> 
  </div>
  {carousel:s}
</body>

<!-- Render Markdown -->

<body>
  <!-- highlight.js: https://highlightjs.org/download/ -->
  <script src="https://cdnjs.cloudflare.com/ajax/libs/highlight.js/11.5.0/highlight.min.js"></script>
  <!-- marked.js -->
  <script src="https://cdn.jsdelivr.net/npm/marked/marked.min.js"></script>
  <script>{script:s}</script>
</body>
</html>
"""
with open("_build/html/index_7days.html", 'w') as fout:
    fout.write(page)

# Debugging papers